In [1]:
#导入一系列包
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
#stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [3]:
file_path2='D:/files/data/stopwordlisteng.TXT'
stopworddic1=stop_words
data=[]
with open(file_path2, "r") as f:
    for line in f.readlines():
        line = line.strip('\n')  #去掉列表中每一个元素的换行符
        data.append(line)
        #print(line)
for i in range(len(data)):
    stopworddic1.append(data[i])

fh=[',','(',')','e','.','g','-']

for i in range(1000):
    fh.append(str(i))

for i in range(len(fh)):
    stopworddic1.append(fh[i])

In [4]:
# Import Dataset
file_path='C:/Users/asus/Desktop/d2v-train.xls'
df=pd.read_excel(file_path)
data = df.Content.values.tolist()

In [5]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

In [6]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [7]:
stop_words = stopworddic1
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [8]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
data_words
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [9]:
type(data_lemmatized)

list

In [11]:
#data_lemmatized[0]
str1=''
for i in range(len(data_lemmatized[0])):
    str1=str1+data_lemmatized[0][i]+' '
str1

'arm techcon san jose usa october leader automotive computing industry announce collaboration plan drive car organization autonomous vehicle compute consortium avcc officially launch arm techcon hold san jose california usa bring industry leader automotive automotive component semiconductor computing industry play lead role support expertise autonomous computing avcc arm bosch continental denso motor nvidia nxp semiconductor toyota expand consortium address scale rollout drive car contribute solve issue '

In [12]:
df1=df
for j in range(len(df1)):
    str1=''
    for i in range(len(data_lemmatized[j])):
        str1=str1+data_lemmatized[j][i]+' '
    df1.loc[j,'Content']=str1

In [13]:
#df1

Content
0     arm techcon san jose usa october leader automo...
1     step achieve avcc vision common goal developme...
2     company understand technical complexity barrie...
3     avcc accept party automotive ecosystem challen...
4     autonomous vehicle compute consortium support ...
5     cost autonomous drive avcc bring competitive m...
6     consortium avcc autonomous vehicle computing c...
7     avcc computing system autonomous drive industr...
8     avcc process separate competitive competitive ...
9     competitive competitive arai protocol car datu...
10    approach competitive theme field avcc recruit ...
11    time establishment avcc confirm company organi...
12    waymo abolishe word drive implicitly criticize...
13    alphabet drive car developer waymo january rem...
14    tesla drive feature criticize mislead waymo stay 
15    car maker misuse wording drive car society con...
16    waymo start drive car development project goog...
17    waymo sister company google alphabet technolog...
18    waymo mention tesla directly term change ironi...
19    change accuracy wording save life waymo blog p...
20    accuracy help distinguish autonomous technolog...
21    tesla release autonomous drive feature public ...
22    company autonomous drive limited drive assista...
23    weimo refer blog january tesla marketing misle...
24    car maker misuse term drive false impression s...
25    false impression pose risk driver safety safet...
26    waymo announce change educational campaign tal...
27    consensus company express autonomous drive tec...
28    tesla ceo musk ambition autonomous driving ach...
29    waymo launch unmanned driving ride_share servi...
...                                                 ...
3022  metal lane recognition apparatus autonomous ve...
3023  distribute system operation autonomous vehicle...
3024  method assist user locate autonomous vehicle c...
3025  method define navigable path network autonomou...
3026  method dynamically control vehicle autonomous ...
3027  collision_avoidance warning system drive vehic...
3028  apparatus select risk rating autonomous vehicl...
3029  road marking_coate autonomous vehicle radar sy...
3030  method control autonomous vehicle involve cont...
3031  unmanned vehicle partial path planning method ...
3032  method generate trajectory autonomous vehicle ...
3033  method crack observation duty watch ahead driv...
3034  method drive autonomous vehicle driverless car...
3035  vehicle cruise control device detect replace d...
3036  exercise method smart exercise system autonomo...
3037  method provide interactive traffic control veh...
3038  brake system partially autonomous vehicle seco...
3039  method object detection detection autonomous v...
3040  transitory_computer readable_storage medium co...
3041  system generating route autonomous vehicle pro...
3042  method operate automate vehicle involve determ...
3043  signal detection control system vehicle autono...
3044  method operate automate vehicle involve determ...
3045  method operate automate vehicle involve receiv...
3046  method recognize roadside_plante involve recei...
3047  emergency method partially automate vehicle in...
3048  control system control brake steering drive au...
3049  autonomous vehicle automatic feeding cow pair ...
3050  method process digital content processor media...
3051  apparatus smartphone autonomous vehicle schedu...

[3052 rows x 1 columns]

In [14]:
dd=[[1.8938395,
  2.011158,
  2.263424,
  1.1542643,
  2.7299583,
  2.2245936,
  2.1777775,
  2.0515175,
  2.0011585,
  3.178644,
  2.1889484,
  1.291473],
 [1.3301122,
  1.7255459,
  1.8380831,
  1.3948087,
  2.4072058,
  2.2162957,
  1.9207909,
  1.6256784,
  1.5512832,
  2.7910209,
  1.9238516,
  1.3528124],
 [1.8629915,
  2.1390946,
  2.7547545,
  1.1400726,
  3.056074,
  2.7857487,
  1.8871542,
  2.2247832,
  1.8484294,
  2.8950088,
  2.4505925,
  1.6903833],
 [1.9278787,
  1.8511995,
  2.5198634,
  1.0686388,
  2.8544493,
  2.636237,
  2.137646,
  2.1319826,
  1.9936819,
  3.119442,
  2.39661,
  1.3459713],
 [1.5007389,
  1.8752766,
  2.1423864,
  1.139084,
  2.4750593,
  2.516314,
  1.6452528,
  1.6204513,
  1.2660694,
  2.4708273,
  2.0237606,
  1.1777643],
 [1.991225,
  2.1582398,
  2.8089988,
  1.2228725,
  2.9558918,
  2.9419727,
  1.7090675,
  2.1644304,
  1.6560578,
  2.7523272,
  2.5111384,
  1.5330634],
 [1.9146111,
  2.0150812,
  2.334152,
  0.9149534,
  2.5111792,
  2.603199,
  1.7231396,
  1.8136587,
  1.7399902,
  2.950314,
  2.0823212,
  1.0490937],
 [2.0675583,
  1.6073687,
  2.3448012,
  1.8185898,
  2.8818762,
  2.3636823,
  2.4444282,
  2.2630942,
  2.2311778,
  3.2139926,
  2.469319,
  1.7929882],
 [1.4988312,
  1.7638683,
  1.9003025,
  1.3035163,
  2.1360617,
  2.4477637,
  1.8577703,
  1.624201,
  1.3339121,
  2.697213,
  2.0185606,
  1.0257411]]

In [15]:
type(dd)

list

In [11]:
test_2 = pd.DataFrame()  
for j in range(12):
    for i in range(12):
        test_2.loc[str(j),i]=0

In [12]:
 test_2

0    1    2    3    4    5    6    7    8    9    10   11
0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
1   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
3   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
4   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
5   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
6   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
7   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
8   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
9   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
10  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
11  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [13]:
mydf=test_2
for i in range(len(dd3)):
    for j in range(len(dd3[i])):
        mydf.loc[str(i),j]=dd3[i][j]

In [14]:
mydf

0         1         2         3         4         5         6   \
0   1.000000  0.858514  0.813243  0.352866  0.648078  0.725385  0.431475   
1   0.858514  1.000000  0.779853  0.320944  0.608579  0.688861  0.265920   
2   0.813243  0.779853  1.000000  0.515458  0.871625  0.800595  0.409036   
3   0.352866  0.320944  0.515458  1.000000  0.517099  0.461833  0.729662   
4   0.648078  0.608579  0.871625  0.517099  1.000000  0.560667  0.485451   
5   0.725385  0.688861  0.800595  0.461833  0.560667  1.000000  0.250764   
6   0.431475  0.265920  0.409036  0.729662  0.485451  0.250764  1.000000   
7   0.800085  0.703129  0.833410  0.685228  0.780894  0.652868  0.704597   
8   0.848332  0.734479  0.728116  0.413084  0.721822  0.537432  0.616390   
9   0.621584  0.515427  0.468628  0.467971  0.435687  0.272398  0.729897   
10  0.542736  0.423142  0.736752  0.689404  0.680392  0.661597  0.606230   
11  0.788177  0.766957  0.838597  0.417277  0.814355  0.686785  0.383277   

          7         8         9         10        11  
0   0.800085  0.848332  0.621584  0.542736  0.788177  
1   0.703129  0.734479  0.515427  0.423142  0.766957  
2   0.833410  0.728116  0.468628  0.736752  0.838597  
3   0.685228  0.413084  0.467971  0.689404  0.417277  
4   0.780894  0.721822  0.435687  0.680392  0.814355  
5   0.652868  0.537432  0.272398  0.661597  0.686785  
6   0.704597  0.616390  0.729897  0.606230  0.383277  
7   1.000000  0.815194  0.699281  0.802965  0.790005  
8   0.815194  1.000000  0.790068  0.553068  0.761960  
9   0.699281  0.790068  1.000000  0.490853  0.406713  
10  0.802965  0.553068  0.490853  1.000000  0.565609  
11  0.790005  0.761960  0.406713  0.565609  1.000000

In [15]:
mydf.to_excel('D:/files/data/DIInew/compare333.xls', index=False)

In [22]:
dd1=[[0.6615587958830341,
  0.6634992975157571,
  0.830629076000617,
  0.458685003719405,
  0.6394927844697487,
  0.9152569092055249,
  0.18581018911277278,
  0.5959469005571834,
  0.48351951713642116,
  0.23473379242915823,
  0.59277676207881,
  0.7489367570640919],
 [0.872431353930749,
  0.7622841542257401,
  0.9105476597362187,
  0.4866807953147721,
  0.7425470632524973,
  0.7981170789595392,
  0.46700676312413747,
  0.7812320081030797,
  0.7356157034778792,
  0.5087756807803545,
  0.7037337672481776,
  0.714919373623493],
 [0.7638400377194682,
  0.6331161931966628,
  0.5547023794269035,
  0.29521249938417055,
  0.42868393475465727,
  0.6252274892425546,
  0.4027617939805247,
  0.4910988158793308,
  0.6356510232746929,
  0.5029985221847217,
  0.4048131810615534,
  0.46809710561462625],
 [0.6837708305082286,
  0.8414316905917018,
  0.7383642115129023,
  0.42154536560553696,
  0.602110533436507,
  0.7284241913547796,
  0.1542432245532162,
  0.5647174947001977,
  0.49288101195311995,
  0.2723304689029609,
  0.4511873274468681,
  0.7514337202574011],
 [0.871166115036009,
  0.7356161551500727,
  0.8765595482762487,
  0.5089083374136902,
  0.7856340773529371,
  0.718233974396977,
  0.6020061465888213,
  0.841813183645195,
  0.9075397471131361,
  0.7364781607187952,
  0.6903889859251753,
  0.8022014721129006],
 [0.6071124840928335,
  0.5735112464787965,
  0.4677650813144205,
  0.3324905988784612,
  0.48753392419431973,
  0.4420726576479145,
  0.5591778527593393,
  0.5214919127391378,
  0.7339025744166419,
  0.5807548356508077,
  0.34788673246552443,
  0.5985928782772263],
 [0.6641694301919095,
  0.6803730038528998,
  0.8243846247777115,
  0.6244081969467157,
  0.8235156636117542,
  0.7074132849084949,
  0.5486276156580607,
  0.7758004640270667,
  0.6784928246046936,
  0.42266699835013355,
  0.6856033660052656,
  0.8985137819949789],
 [0.5761265541347335,
  0.78596720081368,
  0.670779771007897,
  0.35275849758849664,
  0.5060825053823997,
  0.6970318866942771,
  0.2432639594428693,
  0.5076522844699755,
  0.4453025624772593,
  0.3084316053689729,
  0.45948854040381804,
  0.5597634389518117],
 [0.8362282362920044,
  0.7673869611753019,
  0.9387044414101283,
  0.45250826908588687,
  0.9118154795754321,
  0.7220624259566977,
  0.47693478629835767,
  0.8059277087738246,
  0.8445785328969979,
  0.5719659645411129,
  0.6721784044232869,
  0.8529419453512089]]

In [5]:
dd2=[[1.0,
  0.7289841348502514,
  0.5390240965388275,
  0.8050332642123517,
  0.7443484124243588,
  0.41825181221634466,
  0.7996290291520088,
  0.7624174050641229,
  0.7834087057400029],
 [0.7289841348502514,
  1.000000096770844,
  0.7953939824527846,
  0.694281870878825,
  0.8431021937314349,
  0.5611317260144962,
  0.7370736386029821,
  0.6548276404287796,
  0.8759393884993183],
 [0.5390240965388275,
  0.7953939824527846,
  1.0000000735040744,
  0.6152495675869347,
  0.689007249908193,
  0.7673593491629453,
  0.5574222446625403,
  0.6100850987597529,
  0.6569751716190545],
 [0.8050332642123517,
  0.694281870878825,
  0.6152495675869347,
  0.9999999361259739,
  0.6121422913226059,
  0.5626508416771604,
  0.7758698751836152,
  0.7735851297690332,
  0.7125086255961585],
 [0.7443484124243588,
  0.8431021937314349,
  0.689007249908193,
  0.6121422913226059,
  1.0,
  0.668293306455756,
  0.7905144177974691,
  0.6168318060224958,
  0.9417382899875693],
 [0.41825181221634466,
  0.5611317260144962,
  0.7673593491629453,
  0.5626508416771604,
  0.668293306455756,
  1.0,
  0.6949167290673113,
  0.5209080785079048,
  0.6437983978565376],
 [0.7996290291520088,
  0.7370736386029821,
  0.5574222446625403,
  0.7758698751836152,
  0.7905144177974691,
  0.6949167290673113,
  0.9999997988143728,
  0.6967571610173764,
  0.8487374044172096],
 [0.7624174050641229,
  0.6548276404287796,
  0.6100850987597529,
  0.7735851297690332,
  0.6168318060224958,
  0.5209080785079048,
  0.6967571610173764,
  1.0000000652392662,
  0.667474392162117],
 [0.7834087057400029,
  0.8759393884993183,
  0.6569751716190545,
  0.7125086255961585,
  0.9417382899875693,
  0.6437983978565376,
  0.8487374044172096,
  0.667474392162117,
  1.0000000613124769]]

In [6]:
dd3=[[1.0,
  0.8585141757913639,
  0.8132434728574118,
  0.3528661903469807,
  0.6480776377727712,
  0.7253854588471191,
  0.4314745113950524,
  0.8000854871098924,
  0.8483316142032564,
  0.6215837570971469,
  0.5427356260218562,
  0.7881765795588415],
 [0.8585141757913639,
  1.0,
  0.7798529848643866,
  0.3209443184061978,
  0.6085794833875148,
  0.6888607043098789,
  0.26591999258791227,
  0.7031290709791824,
  0.7344789696143571,
  0.515426690802212,
  0.4231418500091626,
  0.7669565429816249],
 [0.8132434728574118,
  0.7798529848643866,
  1.0000000971175038,
  0.515458205734995,
  0.8716245894680605,
  0.8005954316484603,
  0.40903559262061695,
  0.8334098708821455,
  0.7281164588152502,
  0.46862757180185194,
  0.7367516753557969,
  0.838596773400387],
 [0.3528661903469807,
  0.3209443184061978,
  0.515458205734995,
  1.0000001136821466,
  0.5170988722831168,
  0.4618329492495529,
  0.7296621000230056,
  0.6852284697911676,
  0.4130838229408827,
  0.4679710195279551,
  0.6894040992303498,
  0.41727690765029607],
 [0.6480776377727712,
  0.6085794833875148,
  0.8716245894680605,
  0.5170988722831168,
  0.9999999123947807,
  0.560666983913335,
  0.48545053344172806,
  0.7808938955413485,
  0.7218224441927178,
  0.4356871127961434,
  0.6803924641624403,
  0.8143553813055193],
 [0.7253854588471191,
  0.6888607043098789,
  0.8005954316484603,
  0.4618329492495529,
  0.560666983913335,
  1.0,
  0.2507638118370231,
  0.6528682274137306,
  0.537432385948877,
  0.2723984064240568,
  0.6615966423934175,
  0.6867848395359781],
 [0.4314745113950524,
  0.26591999258791227,
  0.40903559262061695,
  0.7296621000230056,
  0.48545053344172806,
  0.2507638118370231,
  0.9999998875791611,
  0.7045968385116862,
  0.6163901967895797,
  0.7298970514626566,
  0.6062295563723635,
  0.3832771170483749],
 [0.8000854871098924,
  0.7031290709791824,
  0.8334098708821455,
  0.6852284697911676,
  0.7808938955413485,
  0.6528682274137306,
  0.7045968385116862,
  1.0000000757065541,
  0.8151943596887444,
  0.6992812896974739,
  0.802964806376979,
  0.7900049629559672],
 [0.8483316142032564,
  0.7344789696143571,
  0.7281164588152502,
  0.4130838229408827,
  0.7218224441927178,
  0.537432385948877,
  0.6163901967895797,
  0.8151943596887444,
  1.0000001818784496,
  0.7900683767097562,
  0.5530678105611739,
  0.7619603076941199],
 [0.6215837570971469,
  0.515426690802212,
  0.46862757180185194,
  0.4679710195279551,
  0.4356871127961434,
  0.2723984064240568,
  0.7298970514626566,
  0.6992812896974739,
  0.7900683767097562,
  1.0000000914061744,
  0.49085333215092947,
  0.40671300359318663],
 [0.5427356260218562,
  0.4231418500091626,
  0.7367516753557969,
  0.6894040992303498,
  0.6803924641624403,
  0.6615966423934175,
  0.6062295563723635,
  0.802964806376979,
  0.5530678105611739,
  0.49085333215092947,
  1.0000001324367804,
  0.5656092892797789],
 [0.7881765795588415,
  0.7669565429816249,
  0.838596773400387,
  0.41727690765029607,
  0.8143553813055193,
  0.6867848395359781,
  0.3832771170483749,
  0.7900049629559672,
  0.7619603076941199,
  0.40671300359318663,
  0.5656092892797789,
  1.0]]